In [1]:
import urllib.request
from bs4 import BeautifulSoup
import os, sys
import hashlib
import numpy as np 
import imageio
import pandas as pd
from PIL import Image
import sys
sys.path.append("..")
from txtProgressBar import printProgressBar2 as print2_
import json
import pickle
import time
#gimbalDownloader

class Downloader(object):
    def __init__(self, username, password, date, directory = None):
        super(Downloader, self).__init__()
        self.username    = username
        self.password    = password
        self.date_str    = date
        self.directory   = directory if directory != None else date
        # self.directoryBMP= self.directory + '/raw_BMPs'
        self.pass_hex    = None
        self.url         = None
        self.numShots    = 0
        self.ID_Shots    = []
        self.dict_whichfiles2download = {}
        self.setWhichFiles({})
        # self.parsed_html = None
    def setWhichFiles(self, tmpdict):
        self.dict_whichfiles2download['origRGB']    = tmpdict['origRGB'] if ("origRGB" in tmpdict) else False
        self.dict_whichfiles2download['json']       = tmpdict['json']    if ("json" in tmpdict) else False
        self.dict_whichfiles2download['rawBMPs']    = tmpdict['rawBMPs'] if ("rawBMPs" in tmpdict) else False
        self.dict_whichfiles2download['CSVfull']    = tmpdict['CSVfull'] if ("CSVfull" in tmpdict) else False
        self.dict_whichfiles2download['CSVcrop']    = tmpdict['CSVcrop'] if ("CSVcrop" in tmpdict) else False
        self.dict_whichfiles2download['CSVspec']    = tmpdict['CSVspec'] if ("CSVspec" in tmpdict) else False
        self.dict_whichfiles2download['RGBwithPOV'] = tmpdict['RGBwithPOV'] if ("RGBwithPOV" in tmpdict) else False
        self.dict_whichfiles2download['BMPwithPOV'] = tmpdict['BMPwithPOV'] if ("BMPwithPOV" in tmpdict) else False
        self.numWhich2down = sum(list(self.dict_whichfiles2download.values())) + (15 if self.dict_whichfiles2download['rawBMPs'] else 0)
    def checkDir(self):
        return os.path.isdir(self.directory)
    def mkdir(self):
        if self.checkDir():
            return True
        else:
            try:
                os.mkdir(self.directory)
                # os.mkdir(self.directoryBMP)
                return True
            except:
                return False
    
    def passConvert(self):
        hash_obj = hashlib.md5(self.password.encode())
        self.pass_hex = hash_obj.hexdigest()
    
    def mkUrl(self):
        self.passConvert()
        self.url = 'http://spectrumcatcher.polarstarspace.com/veggie/report/report.php?id=' + self.username + "&v=103&c=" + self.pass_hex + "&data_day=" + self.date_str
        # print(url)

    def downloadUrl(self):
        self.mkUrl()
        try:
            urllib.request.urlretrieve(self.url, 'tmp1.html')
            return True
        except:
            return False
    def processHTML(self):
        self.mkdir()
        self.downloadUrl()
        html = open('tmp1.html', 'r',  encoding='utf-8').read()
        parsed_html = BeautifulSoup(html, "lxml") #added

        x = parsed_html.body.find('table', attrs={'class':'table table-hover'})
        imgTimes = []
        try:
            for i in range(len(x)-1):
                tmp = str(x.contents[i+1])
                tmp1 = tmp.split("\"")[1].split('_')
                tmp2 = tmp1[-3] + '_'+ tmp1[-2]
                imgTimes.append(tmp2)
                #print(i, '\t',  tmp2)
            self.numShots = len(imgTimes)
            self.ID_Shots = imgTimes
            return True
        except:
            return False
            #probably because of download error or no shots in that day

    def download_1shot(self, each_image_id, i):
        html = open('tmp.tmp', 'r' ,  encoding='utf-8').read()
        parsed_html = BeautifulSoup(html, "lxml") #added
        tmp = parsed_html.body.main.find('div', attrs={'class':'info-title'}).text
        tag = tmp.split(']')[-2].split('[')[1]
        tag = each_image_id.split('_')[1][:2] + 'h_' + tag  # this is also the folder
        #info: angle, time, etc
        x = parsed_html.body.main.div.div.find('div', attrs = {'class':'ndvi-table slider'})
        x.find('div', attrs = {'class':'ndvi-text'})
        tracks = x.find_all('div', {'class':"ndvi-text"})
        infoOfShots = tracks[-2].text
        Pitch   = infoOfShots.split('：')[3][:5]
        Roll    = infoOfShots.split('：')[4][:5]
        Azimuth = infoOfShots.split('：')[5][:5]
        Exposure= infoOfShots.split('：')[7][:5]
        #     print(Pitch, Roll, Azimuth, Exposure, end = '')


        #folder uusgeh gej oroldoh
        if not os.path.isdir(self.directory + "/" + tag):
            os.mkdir(self.directory + "/" + tag)
        
        if not os.path.isdir(self.directory + "/" + tag + '/webGen'):
            os.mkdir(self.directory + "/" + tag + '/webGen')
        

        tmpSubindex = 1
        if self.dict_whichfiles2download['origRGB']:
            print2_(tmpSubindex, self.numWhich2down, i+1, self.numShots, "file", "orig", "\tSet", "Complete", length=self.numWhich2down, length1=20)
            self.download_origRGB(tag, each_image_id)
            tmpSubindex += 1
        if self.dict_whichfiles2download['json']:
            print2_(tmpSubindex, self.numWhich2down, i+1, self.numShots, "file", "json", "\tSet", "Complete", length=self.numWhich2down, length1=20)
            self.download_json(tag, each_image_id)
            tmpSubindex += 1
            
            
        if self.dict_whichfiles2download['rawBMPs']:
            if os.path.isdir(self.directory + "/" + tag + '/rawBMPs'):
                pass
            else:
                os.mkdir(self.directory + "/" + tag + '/rawBMPs')
            for iii in range(16):
                print2_(tmpSubindex, self.numWhich2down, i+1, self.numShots, "file", "bmp{}".format(iii), "\tSet", "Complete", length=self.numWhich2down, length1=20)
                self.download_rawBMPs(tag, each_image_id, iii)
                tmpSubindex += 1
            
        if self.dict_whichfiles2download['CSVfull']:
            print2_(tmpSubindex, self.numWhich2down, i+1, self.numShots, "csv ", "full", "\tSet", "Complete", length=self.numWhich2down, length1=20)
            self.download_CSVfull(tag, each_image_id)
            tmpSubindex += 1
        
        if self.dict_whichfiles2download['CSVcrop']:
            print2_(tmpSubindex, self.numWhich2down, i+1, self.numShots, "csv ", "crop", "\tSet", "Complete", length=self.numWhich2down, length1=20)
            self.download_CSVcrop(tag, each_image_id)
            tmpSubindex += 1
        
        
        if self.dict_whichfiles2download['CSVspec']:
            print2_(tmpSubindex, self.numWhich2down, i+1, self.numShots, "csv ", "spec", "\tSet", "Complete", length=self.numWhich2down, length1=20)
            self.download_CSVspec(tag, each_image_id)
            tmpSubindex += 1
            
        if self.dict_whichfiles2download['RGBwithPOV']:
            print2_(tmpSubindex, self.numWhich2down, i+1, self.numShots, "rgb ", "pov ", "\tSet", "Complete", length=self.numWhich2down, length1=20)
            self.download_RGBwithPOV(tag, each_image_id)
            tmpSubindex += 1
            
        if self.dict_whichfiles2download['BMPwithPOV']:
            print2_(tmpSubindex, self.numWhich2down, i+1, self.numShots, "bmp ", "pov ", "\tSet", "Complete", length=self.numWhich2down, length1=20)
            self.download_BMPwithPOV(tag, each_image_id)
            tmpSubindex += 1


    def download_origRGB(self, tag, each_image_id):
        urlname = "http://spectrumcatcher.polarstarspace.com/veggie/results/"+self.username+"/VeggieCamera_crops_picture_" + each_image_id + ".jpg"
        urllib.request.urlretrieve(urlname,  self.directory + "/" + tag  + '/' + each_image_id + ".jpg")
        
    def download_json(self, tag, each_image_id):
        urlname = 'http://spectrumcatcher.polarstarspace.com/veggie/results/'+self.username+'/VeggieCamera_crops_device_' + each_image_id + ".json"
        urllib.request.urlretrieve(urlname,  self.directory + "/" + tag  + '/' + each_image_id + ".json")
    
    def download_rawBMPs(self, tag, each_image_id, iii):
        urlname = 'http://spectrumcatcher.polarstarspace.com/veggie/results/'+self.username+'/VeggieCamera_crops_spectrum_'+each_image_id+'_'+str(iii)+'.bmp'
        try:
            fName = self.directory + "/" + tag  + '/rawBMPs/' + each_image_id + "_"+str(iii)+".bmp"
            urllib.request.urlretrieve(urlname,  fName)
            im = Image.open(fName)
            tmpy = np.array(im)
            im = Image.fromarray(tmpy.transpose()).transpose(Image.FLIP_LEFT_RIGHT)
            im = Image.fromarray(tmpy.transpose().transpose(Image.FLIP_LEFT_RIGHT))
            im.save(fName)
        except:
            pass
                
    def download_CSVfull(self, tag, each_image_id):
        urlname = 'http://spectrumcatcher.polarstarspace.com/veggie/results/'+self.username+'/' + each_image_id + "_full.csv"
        urllib.request.urlretrieve(urlname,  self.directory + "/" + tag  + '/webGen/' + each_image_id + "_full.csv")
        
    def download_CSVcrop(self, tag, each_image_id):
        urlname = 'http://spectrumcatcher.polarstarspace.com/veggie/results/'+self.username+'/' + each_image_id + "_crop.csv"
        urllib.request.urlretrieve(urlname,  self.directory + "/" + tag  + '/webGen/' + each_image_id + "_crop.csv")
        
    def download_CSVspec(self, tag, each_image_id):
        urlname = 'http://spectrumcatcher.polarstarspace.com/veggie/results/'+self.username+'/' + each_image_id + "_spec_NDVI.csv"
        urllib.request.urlretrieve(urlname,  self.directory + "/" + tag  + '/webGen/' + each_image_id + "_spec_NDVI.csv")
    
    def download_RGBwithPOV(self, tag, each_image_id):
        urlname = 'http://spectrumcatcher.polarstarspace.com/veggie/report/view_photo.php?id='+self.username+'&pic_id=' + each_image_id
        urllib.request.urlretrieve(urlname,  self.directory + "/" + tag  + '/webGen/' + each_image_id + ".jpeg")
        
        
    def download_BMPwithPOV(self, tag, each_image_id):
        urlname = 'http://spectrumcatcher.polarstarspace.com/veggie/results/'+self.username+'/' + each_image_id + "_rotate.png"
        urllib.request.urlretrieve(urlname,  self.directory + "/" + tag  + '/webGen/' + each_image_id + ".png")

    def download_1shot_i(self, i):
        self.download_1shot(self.ID_Shots[i], i)

    def download_set(self):
        for i in range(self.numShots):
            self.download_1shot(self.ID_Shots[i], i)
            


In [2]:
import requests
import sys
from html.parser import HTMLParser

class Copier(object):
    def __init__(self, ip):#, date = None, directory = None):
        self.ip          = ip
        #         self.date_str    = date
        #         self.directory   = directory if directory != None else date
        self.numShots    = 0
        self.ID_Shots    = []
        self.dict_whichfiles2download = {}
        #         self.setWhichFiles({})
        self.parser = self.MyHTMLParser()
        self.allRawFnames = []
        self.allPosibleDates = []
        self.allRawFnamesDict = None
   
        
    def downloadHTML(self):
        url = 'http://{}:8080/storage/emulated/0/Android/data/com.polarstarspace.veggiecamera/files'.format(self.ip)
        r = requests.get(url, allow_redirects=True)
        tmpfile = open('inPhone.html', 'wb')
        tmpfile.write(r.content)
        tmpfile.close()
        
    def processHTML(self, display = None):
        tmpfile = open('inPhone.html', 'r')
        html = tmpfile.read()
        self.parser.feed(html)
        self.allRawFnames, self.allPosibleDates, self.allRawFnamesDict = self.parser.getPhonesDate()
        tmpfile.close()
        print(("All measurement dates in the phone (at {}) are:\n=======================================================\n{}".format(self.ip , self.allRawFnamesDict_str()) if True else ''), end = '')
    
    def allRawFnamesDict_str(self):
        tmp = ''
        for a1date in list(c.allRawFnamesDict.keys()):
            fnamesIna1date = c.allRawFnamesDict[a1date]
            numFiles = len(fnamesIna1date)
            numJsons = len([fname for fname in fnamesIna1date if "json" in fname])
            numJpgs  = len([fname for fname in fnamesIna1date if "jpg" in fname])
            numBMPs  = len([fname for fname in fnamesIna1date if "bmp" in fname])
            tmp_ = "date {}:\t{}files={}jsos +{}jpgs +{}bmps".format(a1date, str(numFiles).rjust(7, ' '), str(numJsons).rjust(5, ' '),str(numJpgs).rjust(5, ' ') , str(numBMPs).rjust(6, ' '))
            tmp += tmp_
        return tmp
    
    def download1day(self, date, directory = None):
        if not (date in c.allRawFnamesDict.keys()):
            print("Error there no files related to day:{}".format(date))
            print("Posssible dates are: {}".format(list(c.allRawFnamesDict.keys())))
        fnames1date = c.allRawFnamesDict[date]
        numFiles = len(fnames1date)
        numJsons = len([fname for fname in fnames1date if "json" in fname])
        numJpgs  = len([fname for fname in fnames1date if "jpg" in fname])
        numBMPs  = len([fname for fname in fnames1date if "bmp" in fname])
        self.tmpdir = directory if directory != None else date
        if not os.path.isdir(self.tmpdir):
            os.mkdir(self.tmpdir)
        self.rawBMPsDir = os.path.join(self.tmpdir, "rawBMPs")
        if not os.path.isdir(self.rawBMPsDir):
            os.mkdir(self.rawBMPsDir)
        countF = 1
        countJ = 1
        countR = 1
        countB = 1
        for fname in fnames1date:
            if   ".json" in fname:
                os.system('wget http://{}:8080/storage/emulated/0/Android/data/com.polarstarspace.veggiecamera/files/VeggieCamera_crops_device_{} -O {}'.format(self.ip, fname, os.path.join(self.tmpdir, fname)))
                os.system("echo \"" +  fname + "\" >> " + os.path.join(self.tmpdir, "id.txt"))
                print2_(countJ, numJsons, countF, numFiles, "{} json: ".format(date), fname, "\tWhole Set", "Complete\t", length=15, length1=15)
                countJ +=1
            elif ".jpg" in fname:
                os.system('wget http://{}:8080/storage/emulated/0/Android/data/com.polarstarspace.veggiecamera/files/VeggieCamera_crops_picture_{} -O {}'.format(self.ip, fname, os.path.join(self.tmpdir, fname) ))
                print2_(countR, numJpgs, countF, numFiles, "{} jpg: ".format(date), fname, "\tWhole  Set", "Complete\t", length=15, length1=15)
                countR +=1
            elif ".bmp" in fname:
                os.system('wget http://{}:8080/storage/emulated/0/Android/data/com.polarstarspace.veggiecamera/files/VeggieCamera_crops_spectrum_{} -O {}'.format(self.ip, fname, os.path.join(self.rawBMPsDir, fname)))
                print2_(countB, numBMPs, countF, numFiles, "{}  bmp: ".format(date), fname, "\tWhole Set", "Complete\t", length=15, length1=15)
                countB +=1
            countF+=1
    class MyHTMLParser(HTMLParser):
        def __init__(self):
            super().__init__()
            self.allPosibleDates = []
            self.allRawFnames = []
            self.lastdate = ''
            self.allRawFnamesDict = {}
            
        def handle_starttag(self, tag, attrs):
            pass
            #print("Encountered a start tag:", tag)

        def handle_endtag(self, tag):
            pass
            #print("Encountered an end tag :", tag)

        def handle_data(self, data):
            #print(data)
            if ('.json' in data) or ('jpg' in data) or ('bmp' in data):
                self.allRawFnames.append(data)
                if ('.json' in data): #example VeggieCamera_crops_device_20201108_170014.json
                    curdate = data.split('_')[3]
                    if curdate != self.lastdate:
                        self.allPosibleDates.append(curdate)
                        self.lastdate = curdate
                        self.allRawFnamesDict[curdate] = [] 
                    self.allRawFnamesDict[curdate].append('{}_{}'.format(data.split('_')[3], data.split('_')[4]))
                if ('.jpg' in data): #example VeggieCamera_crops_picture_20201108_170033.jpg
                    curdate = data.split('_')[3]
                    if curdate != self.lastdate:
                        self.allPosibleDates.append(curdate)
                        self.lastdate = curdate
                        self.allRawFnamesDict[curdate] =[]
                    self.allRawFnamesDict[curdate].append('{}_{}'.format(data.split('_')[3], data.split('_')[4]))
                
                if ('.bmp' in data): #VeggieCamera_crops_spectrum_20201108_164859_0.bmp
                    curdate = data.split('_')[3]
                    if curdate != self.lastdate:
                        self.allPosibleDates.append(curdate)
                        self.lastdate = curdate
                        self.allRawFnamesDict[curdate] = []
                    self.allRawFnamesDict[curdate].append('{}_{}_{}'.format(data.split('_')[3], data.split('_')[4], data.split('_')[5]))
                
                    
        def getPhonesDate(self):
            return self.allRawFnames, self.allPosibleDates, self.allRawFnamesDict

In [ ]:
d = Downloader("mkuriki", "mkuriki", "2020-11-08", "dname1108")
Types = {
    "origRGB"  : True,
    "json"     : True,
    "rawBMPs"  : True,
    "CSVfull"  : True,
    "CSVcrop"  : True,
    "CSVspec"  : True,
    "RGBwithPOV": True,
    "BMPwithPOV": True,
}
d.setWhichFiles(Types)
d.processHTML()
d.download_set()

In [3]:
c = Copier('192.168.11.4')
Types = {
    "origRGB"  : True,
    "json"     : True,
    "rawBMPs"  : True}
# c.setWhichFiles(Types)
c.downloadHTML()
c.processHTML(display = True)

All measurement dates in the phone (at 192.168.11.4) are:
date 20201211:	    612files=   36jsos +   36jpgs +   540bmps

In [5]:
c.download1day('20201211')

20201211  bmp:  |███████████████| 100.0% 20201211_192221_9.bmp 	Whole Set |███████████████| 100.0% Complete	 
